In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy
import sklearn

In [2]:
df_featuresonly = pd.read_excel("wc_data_ready_featuresonly.xlsx", engine="openpyxl")
df_featurestarget = pd.read_excel("wc_data_ready_features_target.xlsx", engine="openpyxl")

In [3]:
df_featurestarget

,Temperature (Celsius),Precipitation (%),Wind Speed (km/h),Atmospheric Pressure (hPa),Visibility (km),Humidity (%),Weather Type,Cloud Cover_clear,Cloud Cover_cloudy,Cloud Cover_overcast,Cloud Cover_partly cloudy,Season_Autumn,Season_Spring,Season_Summer,Season_Winter,Location_coastal,Location_inland,Location_mountain
0,0.363363,0.791792,0.1250,0.153222,-0.333333,0.310904,1,False,False,False,True,False,False,False,True,False,True,False
1,0.930430,0.659660,0.0000,0.180712,1.111111,1.492028,0,False,False,False,True,False,True,False,False,False,True,False
2,0.756757,0.208208,-0.1875,0.509238,0.111111,-0.151275,3,True,False,False,False,False,True,False,False,False,False,True
3,0.920420,0.791792,-0.8750,0.848580,-0.888889,0.824436,3,True,False,False,False,False,True,False,False,True,False,False
4,0.674174,0.610611,1.0625,-0.754845,-0.555556,0.362257,1,False,False,True,False,False,False,False,True,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12313,0.283283,0.659660,0.7500,-0.192429,-0.888889,0.362257,1,False,False,True,False,False,False,True,False,False,False,True
12314,0.174675,0.290290,-0.6250,2.695358,0.222222,0.464964,2,False,True,False,False,False,False,False,True,True,False,False
12315,0.756757,0.321321,-0.3750,0.237494,0.888889,0.516317,0,False,False,True,False,True,False,False,False,True,False,False
12316,0.240741,0.936436,0.1875,-1.043263,-0.666667,0.464964,2,False,False,True,False,False,False,False,True,False,True,False


In [4]:
from sklearn.model_selection import train_test_split
X = df_featuresonly
y = df_featurestarget["Weather Type"]

X_train, X_temp, y_train, y_temp = train_test_split(X, y, train_size=0.8, random_state=40)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=40)


In [5]:
# Ok so the multi_class parameter is deprecated. I think the function automatically selects multinomial LogReg when 
# number of classes (in target, right) >= 3. That's what the documentation seems to say 

from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=40)
model.fit(X_train, y_train)

LogisticRegression(random_state=40)